In [ ]:
import pandas as pd
import rioxarray as rxr
import rasterio
from rasterio.mask import mask
from pyproj import Transformer, CRS
from shapely.geometry import Point, mapping
import numpy as np
import torch
from tqdm import tqdm

def extract_buffered_satellite_data(tiff_path, csv_path, buffer_distance=50, use_gpu=True):
    """
    Extracts the average band values within a specified buffer (in meters) 
    around each coordinate from a GeoTIFF using Mac MPS (GPU) acceleration.
    
    Parameters:
        tiff_path (str): Path to the GeoTIFF file.
        csv_path (str): Path to the CSV file containing Latitude and Longitude.
        buffer_distance (float): Buffer radius in meters.
        use_gpu (bool): Whether to use GPU (MPS) acceleration.
    
    Returns:
        pd.DataFrame: DataFrame with average band values within the buffer.
    """
    device = torch.device("mps" if torch.backends.mps.is_available() and use_gpu else "cpu")
    print(f"Using device: {device}")

    dataset = rxr.open_rasterio(tiff_path)
    tiff_crs = dataset.rio.crs
    print(f"Original TIFF CRS: {tiff_crs}")

    if not tiff_crs.is_projected:
        print("Reprojecting TIFF to UTM for correct buffering...")
        reprojected_tiff_path = "reprojected.tif"
        dataset = dataset.rio.reproject(dst_crs=CRS.from_epsg(32618))  # Example UTM Zone 18N
        dataset.rio.to_raster(reprojected_tiff_path)
        tiff_path = reprojected_tiff_path  # Use the reprojected file
        tiff_crs = dataset.rio.crs
        print(f"New TIFF CRS after reprojection: {tiff_crs}")

    df = pd.read_csv(csv_path)
    latitudes = torch.tensor(df['Latitude'].values, dtype=torch.float32, device=device)
    longitudes = torch.tensor(df['Longitude'].values, dtype=torch.float32, device=device)

    transformer = Transformer.from_crs("EPSG:4326", tiff_crs, always_xy=True)
    coords_np = np.vstack([longitudes.cpu().numpy(), latitudes.cpu().numpy()])
    transformed_coords = np.array(transformer.transform(coords_np[0], coords_np[1]))

    num_bands = dataset.shape[0]
    band_values = {f'B{band+1:02d}': torch.full((len(df),), float('nan'), dtype=torch.float32, device=device) for band in range(num_bands)}

    with rasterio.open(tiff_path) as src:
        raster_bounds = src.bounds
        print(f"Raster bounds: {raster_bounds}")

        for idx, (x, y) in tqdm(enumerate(zip(transformed_coords[0], transformed_coords[1])),
                                 total=len(latitudes), desc="Extracting values"):
            if not (raster_bounds.left <= x <= raster_bounds.right and raster_bounds.bottom <= y <= raster_bounds.top):
                print(f"Skipping point {idx} ({x:.2f}, {y:.2f}) - Outside raster bounds")
                continue
            
            point = Point(x, y).buffer(buffer_distance)
            geojson_geom = [mapping(point)]

            try:
                out_image, _ = mask(src, geojson_geom, crop=True)
                out_image = torch.tensor(out_image, dtype=torch.float32, device=device) 

                for band in range(out_image.shape[0]):
                    band_values[f'B{band+1:02d}'][idx] = torch.nanmean(out_image[band])

            except Exception as e:
                print(f"Skipping point {idx} ({y:.2f}, {x:.2f}) due to error: {e}")

    band_df = pd.DataFrame({key: band_values[key].cpu().numpy() for key in band_values})

    return band_df

tiff = '/Users/ibolam/Projects/Abroad/01. UMD/25SPRING_DATA605/InfoChallege/UMD_IC25_Participant_Package/S2_alloutput_IC25.tiff'
trd = '/Users/ibolam/Projects/Abroad/01. UMD/25SPRING_DATA605/InfoChallege/UMD_IC25_Participant_Package/Training_Data_IC25.csv'

# ✅ 8️⃣ Run with Mac MPS GPU Acceleration
final_data = extract_buffered_satellite_data(tiff, trd, buffer_distance=50, use_gpu=True)

# ✅ 9️⃣ Save results
final_data.to_csv("output50.csv", index=False)

Using device: mps
Original TIFF CRS: EPSG:4326
Reprojecting TIFF to UTM for correct buffering...
New TIFF CRS after reprojection: EPSG:32618
Raster bounds: BoundingBox(left=297192.3140653926, bottom=4311830.711916517, right=336053.61640117853, top=4347129.585479343)


Extracting values: 100%|██████████| 34502/34502 [01:30<00:00, 379.25it/s]


In [2]:
# ✅ 8️⃣ Run with Mac MPS GPU Acceleration
final_data = extract_buffered_satellite_data(tiff, trd, buffer_distance=100, use_gpu=True)

# ✅ 9️⃣ Save results
final_data.to_csv("output100.csv", index=False)

Using device: mps
Original TIFF CRS: EPSG:4326
Reprojecting TIFF to UTM for correct buffering...
New TIFF CRS after reprojection: EPSG:32618
Raster bounds: BoundingBox(left=297192.3140653926, bottom=4311830.711916517, right=336053.61640117853, top=4347129.585479343)


Extracting values: 100%|██████████| 34502/34502 [01:36<00:00, 356.59it/s]


In [4]:
# ✅ 8️⃣ Run with Mac MPS GPU Acceleration
final_data = extract_buffered_satellite_data(tiff, trd, buffer_distance=150, use_gpu=True)

# ✅ 9️⃣ Save results
final_data.to_csv("output150.csv", index=False)

Using device: mps
Original TIFF CRS: EPSG:4326
Reprojecting TIFF to UTM for correct buffering...
New TIFF CRS after reprojection: EPSG:32618
Raster bounds: BoundingBox(left=297192.3140653926, bottom=4311830.711916517, right=336053.61640117853, top=4347129.585479343)


Extracting values: 100%|██████████| 34502/34502 [01:41<00:00, 340.54it/s]


In [3]:
# ✅ 8️⃣ Run with Mac MPS GPU Acceleration
final_data = extract_buffered_satellite_data(tiff, trd, buffer_distance=200, use_gpu=True)

# ✅ 9️⃣ Save results
final_data.to_csv("output200.csv", index=False)

Using device: mps
Original TIFF CRS: EPSG:4326
Reprojecting TIFF to UTM for correct buffering...
New TIFF CRS after reprojection: EPSG:32618
Raster bounds: BoundingBox(left=297192.3140653926, bottom=4311830.711916517, right=336053.61640117853, top=4347129.585479343)


Extracting values: 100%|██████████| 34502/34502 [01:45<00:00, 326.65it/s]
